In [1]:
import os
import cv2
import numpy as np
import ipywidgets as ipyw
import lz4.frame
import pandas as pd

import matplotlib

from matplotlib import pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [12]:
class ImageSliceViewer3D:
    """ 
    https://github.com/mohakpatel/ImageSliceViewer3D/blob/master/ImageSliceViewer3D.ipynb
    """ 
    def __init__(self, volume, figsize=(8,8), cmap='gray'):
        self.volume = volume
        self.figsize = figsize
        self.cmap = cmap
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane selection
        orient = {"y-z":[1,2,0], "z-x":[2,0,1], "x-y": [0,1,2]}
        self.vol = np.transpose(self.volume, orient[view])
        maxZ = self.vol.shape[2] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        self.fig = plt.figure(figsize=self.figsize)
        plt.imshow(self.vol[:,:,z], cmap=plt.get_cmap(self.cmap), 
            vmin=self.v[0], vmax=self.v[1])

In [120]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
cfr_meta_dir = os.path.join(cfr_data_root, 'metadata_200131')
cfr_meta_file = '210_getStressTest_files_dset_BWH_200131.parquet'
meta_df = pd.read_parquet(os.path.join(cfr_meta_dir, cfr_meta_file))

# Cleaning meta data
rows0 = meta_df.shape[0]
# We need views: remove all rows that do not have view classification
meta_df = meta_df.loc[~meta_df.max_view.isnull()]
# We also need frame_interval
meta_df = meta_df.loc[~meta_df.frame_time.isnull()]
print('Removed {} rows.'.format(rows0 - meta_df.shape[0]))

view_list = sorted(list(meta_df.max_view.unique()))
mode_list = sorted(list(meta_df.dset.unique()))
# LOOP 1: VIEWS
view = view_list[2]
# LOOP 2: MODE
mode = mode_list[2]
# Filter view and mode. Shuffle.
df = meta_df[(meta_df.max_view == view) & (meta_df.dset == mode)].sample(frac = 1)

# LOOP 3: FILES loop over all file names
filename = df.filename.iloc[100]
ser = df.loc[df.filename == filename, :]
dir = ser.dir.values[0]
file = os.path.join(dir, filename)
frame_time = ser.frame_time.values[0]*1e-3
frame_rate = 1/frame_time

try:
    with lz4.frame.open(file, 'rb') as fp:
        data = np.squeeze(np.load(fp))
except IOError as err:
    print('Could not open this file: {}\n {}'.format(file, err))

print('Shape of image array: {}'.format(data.shape))
print('Frame time {} ms -> rate {} Hz.'.format(frame_time, frame_rate))
im = data[10, :, :]

Removed 9 rows.
Shape of image array: (80, 600, 800)
Frame time 0.033333 ms -> rate 30.000300003000028 Hz.


In [121]:
# openCV requires a single-channel 8-bit image
def imscale(im):
    # We can do other things here: e.g. background subtraction or contrast enhancement
    im_scaled = np.uint8((im - np.amin(im))/(np.amax(im) - np.amin(im))*256)
    #im_scaled_eq = im_scaled
    im_scaled_eq = cv2.equalizeHist(im_scaled) 
    return im_scaled_eq

im_list = [imscale(data[im]) for im in range(data.shape[0])]
im_array = np.array(im_list, dtype = np.uint16)
im_array = np.moveaxis(im_array, 0, -1)
print(im_array.shape)

(600, 800, 80)


In [122]:
# subsampling to have the same frame rate

def subsample_time_index_list(frame_time, default_rate, min_frames):
    '''rate: data frame rate,
       default_rate: desired frame rate,
       n_frames: number frames in the default rate (30)'''

    default_times = np.arange(0, min_frames, 1) / default_rate
    times = np.arange(0, default_times[-1] + frame_time, frame_time)
    time_index_list = [np.argmin(np.abs(times - t)) for t in default_times]
    
    return time_index_list

def subsample_video(image_array, frame_time, default_rate=25, min_frames=40):

    convert_video = True
    rate = 1/frame_time
    # Check if the video is long enough
    min_video_len = min_frames/default_rate
    video_len = image_array.shape[-1] / rate
    if (min_video_len <= video_len) & (default_rate < rate):
        #print('Video is long enough and the rate is good.')
        # Get the frame index list
        time_index_list = subsample_time_index_list(frame_time = frame_time,
                                                    default_rate = default_rate,
                                                    min_frames=min_frames)
        # Select the frames from the video
        image_array = image_array[:, :, time_index_list]
    else:
        print('Video is too short or the rate is too low.')
        convert_video = False
    
    return convert_video, image_array

In [123]:
convert_video, image_array = subsample_video(im_array, frame_time)
print(convert_video)
print(image_array.shape)

True
(600, 800, 40)


In [124]:
ImageSliceViewer3D(image_array)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…